<a href="https://colab.research.google.com/github/danzerzine/seospider-colab/blob/main/Running_screamingfrog_SEO_spider_in_Colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Запуск SEO бота Screaming Frog SEO spider в облаке через Google Colab

-------------

> *Protip: под задачу для крупного сайта лучше всего подходят High RAM (25GB) инстансы без GPU/TPU, доступные в PRO подписке*


###Косметическое улучшение: добавляем перенос строки для длинных однострочных команд 



In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

###Подключаем Google Drive в котором хранятся конфиги бота и куда будут сохраняться результаты обхода 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Узнаем внешний IP инстанса 
чтобы затем ручками добавить его в исключения файерволла cloudflare -- иначе очень быстро упремся в rate limit и нам начнут показывать страницу с проверкой на человекообразность

In [ ]:
!wget -qO- http://ipecho.net/plain | xargs echo && wget -qO - icanhazip.com

35.237.19.193
35.237.19.193


###Устанавливаем последнюю версию seo spider, делаем мелкие дела по хозяйству
* Обновляем установленные linux пакеты 
* Копируем настройки с десктопной версии SEO spider в локальную папку инстанса (это нужно чтобы передать токены авторизации к google search console, GA и так далее) 

In [ ]:
#@title Settings directory on GDrive { vertical-output: true, display-mode: "both" }
settings_path = "" #@param {type:"string"}
!wget https://download.screamingfrog.co.uk/products/seo-spider/screamingfrogseospider_16.3_all.deb
!apt-get install screamingfrogseospider_16.3_all.deb
!sudo apt-get update && sudo apt-get upgrade -y
!mkdir -p ~/.ScreamingFrogSEOSpider
!cp -r $settings_path/* ~/.ScreamingFrogSEOSpider

### Запускаем bash скрипт для донастройки инстанса и бота 
Он добавит виртуальный дисплей для вывода из JAVA, переключит бота в режим сохранения результатов на диске вместо RAM и т.д.

In [ ]:
!wget https://raw.githubusercontent.com/fili/screaming-frog-on-google-compute-engine/master/gce-sf.sh -O install.sh && chmod +x install.sh && source ./install.sh

###Делаем симлинк скрытой папки с временными файлами и настройками бота
на случай если придется что-то редактировать или вынимать оттуда наживую, иначе ее не будет видно в браузере файлов слева

In [ ]:
!ln -s ~/.ScreamingFrogSEOSpider ~/ScreamingFrogSEOSpider

###Даем команду боту в headless режиме 
прописываем все нужные флаги для экспорта, настроек, отчетов, выгрузок и так далее

In [ ]:
#@title Crawl settings { vertical-output: true }
url_start = "" #@param {type:"string"}
use_gcs = "" #@param ["", "--use-google-search-console \"account \""] {allow-input: true}
config_path = "" #@param {type:"string"}
output_folder = "" #@param {type:"string"}

!screamingfrogseospider --crawl "$url_start" $use_gcs --headless  --config "$config_path" --output-folder "$output_folder" --timestamped-output --save-crawl --export-tabs "Internal:All,Response Codes:All,Response Codes:Blocked by Robots.txt,Response Codes:Blocked Resource,Response Codes:No Response,Response Codes:Redirection (3xx),Response Codes:Redirection (JavaScript),Response Codes:Redirection (Meta Refresh),Response Codes:Client Error (4xx),Response Codes:Server Error (5xx),Page Titles:All,Page Titles:Missing,Page Titles:Duplicate,Page Titles:Over X Characters,Page Titles:Below X Characters,Page Titles:Over X Pixels,Page Titles:Below X Pixels,Page Titles:Same as H1,Page Titles:Multiple,Meta Description:All,Meta Description:Missing,Meta Description:Duplicate,Meta Description:Over X Characters,Meta Description:Below X Characters,Meta Description:Over X Pixels,Meta Description:Below X Pixels,Meta Description:Multiple,Meta Keywords:All,Meta Keywords:Missing,Meta Keywords:Duplicate,Meta Keywords:Multiple,Canonicals:All,Canonicals:Contains Canonical,Canonicals:Self Referencing,Canonicals:Canonicalised,Canonicals:Missing,Canonicals:Multiple,Canonicals:Non-Indexable Canonical,Directives:All,Directives:Index,Directives:Noindex,Directives:Follow,Directives:Nofollow,Directives:None,Directives:NoArchive,Directives:NoSnippet,Directives:Max-Snippet,Directives:Max-Image-Preview,Directives:Max-Video-Preview,Directives:NoODP,Directives:NoYDIR,Directives:NoImageIndex,Directives:NoTranslate,Directives:Unavailable_After,Directives:Refresh,AMP:All,AMP:Non-200 Response,AMP:Missing Non-AMP Return Link,AMP:Missing Canonical to Non-AMP,AMP:Non-Indexable Canonical,AMP:Indexable,AMP:Non-Indexable,AMP:Missing <html amp> Tag,AMP:Missing/Invalid <!doctype html> Tag,AMP:Missing <head> Tag,AMP:Missing <body> Tag,AMP:Missing Canonical,AMP:Missing/Invalid <meta charset> Tag,AMP:Missing/Invalid <meta viewport> Tag,AMP:Missing/Invalid AMP Script,AMP:Missing/Invalid AMP Boilerplate,AMP:Contains Disallowed HTML,AMP:Other Validation Errors,Structured Data:All,Structured Data:Contains Structured Data,Structured Data:Missing,Structured Data:Validation Errors,Structured Data:Validation Warnings,Structured Data:Parse Errors,Structured Data:Microdata URLs,Structured Data:JSON-LD URLs,Structured Data:RDFa URLs,Sitemaps:All,Sitemaps:URLs in Sitemap,Sitemaps:URLs not in Sitemap,Sitemaps:Orphan URLs,Sitemaps:Non-Indexable URLs in Sitemap,Sitemaps:URLs in Multiple Sitemaps,Sitemaps:XML Sitemap with over 50k URLs,Sitemaps:XML Sitemap over 50MB" --bulk-export "Canonicals:Contains Canonical Inlinks,Canonicals:Self Referencing Inlinks,Canonicals:Canonicalised Inlinks,Canonicals:Missing Inlinks,Canonicals:Multiple Inlinks,Canonicals:Non-Indexable Canonical Inlinks,AMP:All Inlinks,AMP:Non-200 Response Inlinks,AMP:Missing Non-AMP Return Link Inlinks,AMP:Missing Canonical to Non-AMP Inlinks,AMP:Non-Indexable Canonical Inlinks,AMP:Indexable Inlinks,AMP:Non-Indexable Inlinks,Structured Data:Contains Structured Data,Structured Data:Validation Errors,Structured Data:Validation Warnings,Structured Data:JSON-LD URLs,Structured Data:Microdata URLs,Structured Data:RDFa URLs,Sitemaps:URLs in Sitemap Inlinks,Sitemaps:Orphan URLs Inlinks,Sitemaps:Non-Indexable URLs in Sitemap Inlinks,Sitemaps:URLs in Multiple Sitemaps Inlinks" --save-report "Crawl Overview,Redirects:All Redirects,Redirects:Redirect Chains,Redirects:Redirect & Canonical Chains,Canonicals:Canonical Chains,Canonicals:Non-Indexable Canonicals,Pagination:Non-200 Pagination URLs,Pagination:Unlinked Pagination URLs,Hreflang:All hreflang URLs,Hreflang:Non-200 hreflang URLs,Hreflang:Unlinked hreflang URLs,Hreflang:Missing Return Links,Hreflang:Inconsistent Language & Region Return Links,Hreflang:Non Canonical Return Links,Hreflang:Noindex Return Links,Insecure Content,SERP Summary,Orphan Pages,Structured Data:Validation Errors & Warnings Summary,Structured Data:Validation Errors & Warnings,Structured Data:Google Rich Results Features Summary,Structured Data:Google Rich Results Features,HTTP Headers:HTTP Header Summary,Cookies:Cookie Summary" --export-format xlsx --export-custom-summary "Site Crawled,Date,Time,Total URLs Encountered,Total URLs Crawled,Total Internal blocked by robots.txt,Total External blocked by robots.txt,URLs Displayed,Total Internal URLs,Total External URLs,Total Internal Indexable URLs,Total Internal Non-Indexable URLs,JavaScript:All,JavaScript:Uses Old AJAX Crawling Scheme URLs,JavaScript:Uses Old AJAX Crawling Scheme Meta Fragment Tag,JavaScript:Page Title Only in Rendered HTML,JavaScript:Page Title Updated by JavaScript,JavaScript:H1 Only in Rendered HTML,JavaScript:H1 Updated by JavaScript,JavaScript:Meta Description Only in Rendered HTML,JavaScript:Meta Description Updated by JavaScript,JavaScript:Canonical Only in Rendered HTML,JavaScript:Canonical Mismatch,JavaScript:Noindex Only in Original HTML,JavaScript:Nofollow Only in Original HTML,JavaScript:Contains JavaScript Links,JavaScript:Contains JavaScript Content,JavaScript:Pages with Blocked Resources,H1:All,H1:Missing,H1:Duplicate,H1:Over X Characters,H1:Multiple,H2:All,H2:Missing,H2:Duplicate,H2:Over X Characters,H2:Multiple,Internal:All,Internal:HTML,Internal:JavaScript,Internal:CSS,Internal:Images,Internal:PDF,Internal:Flash,Internal:Other,Internal:Unknown,External:All,External:HTML,External:JavaScript,External:CSS,External:Images,External:PDF,External:Flash,External:Other,External:Unknown,AMP:All,AMP:Non-200 Response,AMP:Missing Non-AMP Return Link,AMP:Missing Canonical to Non-AMP,AMP:Non-Indexable Canonical,AMP:Indexable,AMP:Non-Indexable,AMP:Missing <html amp> Tag,AMP:Missing/Invalid <!doctype html> Tag,AMP:Missing <head> Tag,AMP:Missing <body> Tag,AMP:Missing Canonical,AMP:Missing/Invalid <meta charset> Tag,AMP:Missing/Invalid <meta viewport> Tag,AMP:Missing/Invalid AMP Script,AMP:Missing/Invalid AMP Boilerplate,AMP:Contains Disallowed HTML,AMP:Other Validation Errors,Canonicals:All,Canonicals:Contains Canonical,Canonicals:Self Referencing,Canonicals:Canonicalised,Canonicals:Missing,Canonicals:Multiple,Canonicals:Non-Indexable Canonical,Content:All,Content:Spelling Errors,Content:Grammar Errors,Content:Near Duplicates,Content:Exact Duplicates,Content:Low Content Pages,Custom Extraction:All,Custom Search:All,Directives:All,Directives:Index,Directives:Noindex,Directives:Follow,Directives:Nofollow,Directives:None,Directives:NoArchive,Directives:NoSnippet,Directives:Max-Snippet,Directives:Max-Image-Preview,Directives:Max-Video-Preview,Directives:NoODP,Directives:NoYDIR,Directives:NoImageIndex,Directives:NoTranslate,Directives:Unavailable_After,Directives:Refresh,Analytics:All,Analytics:Sessions Above 0,Analytics:Bounce Rate Above 70%,Analytics:No GA Data,Analytics:Non-Indexable with GA Data,Analytics:Orphan URLs,Search Console:All,Search Console:Clicks Above 0,Search Console:No GSC Data,Search Console:Non-Indexable with GSC Data,Search Console:Orphan URLs,Hreflang:All,Hreflang:Contains hreflang,Hreflang:Non-200 hreflang URLs,Hreflang:Unlinked hreflang URLs,Hreflang:Missing Return Links,Hreflang:Inconsistent Language & Region Return Links,Hreflang:Non-Canonical Return Links,Hreflang:Noindex Return Links,Hreflang:Incorrect Language & Region Codes,Hreflang:Multiple Entries,Hreflang:Missing Self Reference,Hreflang:Not Using Canonical,Hreflang:Missing X-Default,Hreflang:Missing,Images:All,Images:Over X KB,Images:Missing Alt Text,Images:Missing Alt Attribute,Images:Alt Text Over X Characters,Link Metrics:All,Meta Description:All,Meta Description:Missing,Meta Description:Duplicate,Meta Description:Over X Characters,Meta Description:Below X Characters,Meta Description:Over X Pixels,Meta Description:Below X Pixels,Meta Description:Multiple,Meta Keywords:All,Meta Keywords:Missing,Meta Keywords:Duplicate,Meta Keywords:Multiple,PageSpeed:All,PageSpeed:Eliminate Render-Blocking Resources,PageSpeed:Defer Offscreen Images,PageSpeed:Efficiently Encode Images,PageSpeed:Properly Size Images,PageSpeed:Minify CSS,PageSpeed:Minify JavaScript,PageSpeed:Reduce Unused CSS,PageSpeed:Reduce Unused JavaScript,PageSpeed:Serve Images in Next-Gen Formats,PageSpeed:Enable Text Compression,PageSpeed:Preconnect to Required Origins,PageSpeed:Reduce Server Response Times (TTFB),PageSpeed:Avoid Multiple Page Redirects,PageSpeed:Preload Key Requests,PageSpeed:Use Video Formats for Animated Content,PageSpeed:Avoid Excessive DOM Size,PageSpeed:Reduce JavaScript Execution Time,PageSpeed:Serve Static Assets with an Efficient Cache Policy,PageSpeed:Minimize Main-Thread Work,PageSpeed:Ensure Text Remains Visible During Webfont Load,PageSpeed:Image Elements Do Not Have Explicit Width & Height,PageSpeed:Avoid Large Layout Shifts,PageSpeed:Avoid Serving Legacy JavaScript to Modern Browsers,PageSpeed:Request Errors,Pagination:All,Pagination:Contains Pagination,Pagination:First Page,Pagination:Paginated 2+ Pages,Pagination:Pagination URL Not in Anchor Tag,Pagination:Non-200 Pagination URLs,Pagination:Unlinked Pagination URLs,Pagination:Non-Indexable,Pagination:Multiple Pagination URLs,Pagination:Pagination Loop,Pagination:Sequence Error,Response Codes:All,Response Codes:Blocked by Robots.txt,Response Codes:Blocked Resource,Response Codes:No Response,Response Codes:Success (2xx),Response Codes:Redirection (3xx),Response Codes:Redirection (JavaScript),Response Codes:Redirection (Meta Refresh),Response Codes:Client Error (4xx),Response Codes:Server Error (5xx),Security:All,Security:HTTP URLs,Security:HTTPS URLs,Security:Mixed Content,Security:Form URL Insecure,Security:Form on HTTP URL,Security:Unsafe Cross-Origin Links,Security:Missing HSTS Header,Security:Bad Content Type,Security:Missing X-Content-Type-Options Header,Security:Missing X-Frame-Options Header,Security:Protocol-Relative Resource Links,Security:Missing Content-Security-Policy Header,Security:Missing Secure Referrer-Policy Header,Sitemaps:All,Sitemaps:URLs in Sitemap,Sitemaps:URLs not in Sitemap,Sitemaps:Orphan URLs,Sitemaps:Non-Indexable URLs in Sitemap,Sitemaps:URLs in Multiple Sitemaps,Sitemaps:XML Sitemap with over 50k URLs,Sitemaps:XML Sitemap over 50MB,Structured Data:All,Structured Data:Contains Structured Data,Structured Data:Missing,Structured Data:Validation Errors,Structured Data:Validation Warnings,Structured Data:Parse Errors,Structured Data:Microdata URLs,Structured Data:JSON-LD URLs,Structured Data:RDFa URLs,Page Titles:All,Page Titles:Missing,Page Titles:Duplicate,Page Titles:Over X Characters,Page Titles:Below X Characters,Page Titles:Over X Pixels,Page Titles:Below X Pixels,Page Titles:Same as H1,Page Titles:Multiple,URL:All,URL:Non ASCII Characters,URL:Underscores,URL:Uppercase,URL:Parameters,URL:Over X Characters,URL:Multiple Slashes,URL:Repetitive Path,URL:Contains Space,URL:Broken Bookmark,URL:Internal Search,Depth 1,Depth 2,Depth 3,Depth 4,Depth 5,Depth 6,Depth 7,Depth 8,Depth 9,Depth 10+,Top Inlinks 1 URL,Top Inlinks 1 Number of Inlinks,Top Inlinks 2 URL,Top Inlinks 2 Number of Inlinks,Top Inlinks 3 URL,Top Inlinks 3 Number of Inlinks,Top Inlinks 4 URL,Top Inlinks 4 Number of Inlinks,Top Inlinks 5 URL,Top Inlinks 5 Number of Inlinks,Top Inlinks 6 URL,Top Inlinks 6 Number of Inlinks,Top Inlinks 7 URL,Top Inlinks 7 Number of Inlinks,Top Inlinks 8 URL,Top Inlinks 8 Number of Inlinks,Top Inlinks 9 URL,Top Inlinks 9 Number of Inlinks,Top Inlinks 10 URL,Top Inlinks 10 Number of Inlinks,Top Inlinks 11 URL,Top Inlinks 11 Number of Inlinks,Top Inlinks 12 URL,Top Inlinks 12 Number of Inlinks,Top Inlinks 13 URL,Top Inlinks 13 Number of Inlinks,Top Inlinks 14 URL,Top Inlinks 14 Number of Inlinks,Top Inlinks 15 URL,Top Inlinks 15 Number of Inlinks,Top Inlinks 16 URL,Top Inlinks 16 Number of Inlinks,Top Inlinks 17 URL,Top Inlinks 17 Number of Inlinks,Top Inlinks 18 URL,Top Inlinks 18 Number of Inlinks,Top Inlinks 19 URL,Top Inlinks 19 Number of Inlinks,Top Inlinks 20 URL,Top Inlinks 20 Number of Inlinks,Response Times 0s to 1s,Response Times 1s to 2s,Response Times 2s to 3s,Response Times 3s to 4s,Response Times 4s to 5s,Response Times 5s to 6s,Response Times 6s to 7s,Response Times 7s to 8s,Response Times 8s to 9s,Response Times 10s or more" 

# ✦ *Colab Still Alive Console Script:*
<p><font size=2px ><font color="red"> Tip - Set a javascript interval to click on the connect button every 60 seconds. Open developer-settings (in your web-browser) with Ctrl+Shift+I then click on console tab and type this on the console prompt. (for mac press Option+Command+I)</font></p><b>Copy script in hidden cell and paste at your browser console !!! DO NOT CLOSE YOUR BROWSER IN ORDER TO STILL RUNNING SCRIPT</b>

<code>function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-connect-button").click() 
}setInterval(ClickConnect,6000)</code>

# *Что в итоге*
На выходе в идеале получаем 
папку с датой обхода и следующими выгрузками в формате Excel


**Tabs**:

```
Internal:All
Response Codes:All
Response Codes:Blocked by Robots.txt
Response Codes:Blocked Resource
Response Codes:No Response
Response Codes:Redirection (3xx)
Response Codes:Redirection (JavaScript)
Response Codes:Redirection (Meta Refresh)
Response Codes:Client Error (4xx)
Response Codes:Server Error (5xx)
Page Titles:All
Page Titles:Missing
Page Titles:Duplicate
Page Titles:Over X Characters
Page Titles:Below X Characters
Page Titles:Over X Pixels
Page Titles:Below X Pixels
Page Titles:Same as H1
Page Titles:Multiple
Meta Description:All
Meta Description:Missing
Meta Description:Duplicate
Meta Description:Over X Characters
Meta Description:Below X Characters
Meta Description:Over X Pixels
Meta Description:Below X Pixels
Meta Description:Multiple
Meta Keywords:All
Meta Keywords:Missing
Meta Keywords:Duplicate
Meta Keywords:Multiple
Canonicals:All
Canonicals:Contains Canonical
Canonicals:Self Referencing
Canonicals:Canonicalised
Canonicals:Missing
Canonicals:Multiple
Canonicals:Non-Indexable Canonical
Directives:All
Directives:Index
Directives:Noindex
Directives:Follow
Directives:Nofollow
Directives:None
Directives:NoArchive
Directives:NoSnippet
Directives:Max-Snippet
Directives:Max-Image-Preview
Directives:Max-Video-Preview
Directives:NoODP
Directives:NoYDIR
Directives:NoImageIndex
Directives:NoTranslate
Directives:Unavailable_After
Directives:Refresh
AMP:All
AMP:Non-200 Response
AMP:Missing Non-AMP Return Link
AMP:Missing Canonical to Non-AMP
AMP:Non-Indexable Canonical
AMP:Indexable
AMP:Non-Indexable
AMP:Missing <html amp> Tag
AMP:Missing/Invalid <!doctype html> Tag
AMP:Missing <head> Tag
AMP:Missing <body> Tag
AMP:Missing Canonical
AMP:Missing/Invalid <meta charset> Tag
AMP:Missing/Invalid <meta viewport> Tag
AMP:Missing/Invalid AMP Script
AMP:Missing/Invalid AMP Boilerplate
AMP:Contains Disallowed HTML
AMP:Other Validation Errors
Structured Data:All
Structured Data:Contains Structured Data
Structured Data:Missing
Structured Data:Validation Errors
Structured Data:Validation Warnings
Structured Data:Parse Errors
Structured Data:Microdata URLs
Structured Data:JSON-LD URLs
Structured Data:RDFa URLs
Sitemaps:All
Sitemaps:URLs in Sitemap
Sitemaps:URLs not in Sitemap
Sitemaps:Orphan URLs
Sitemaps:Non-Indexable URLs in Sitemap
Sitemaps:URLs in Multiple Sitemaps
Sitemaps:XML Sitemap with over 50k URLs
Sitemaps:XML Sitemap over 50MB" --bulk-export "Canonicals:Contains Canonical Inlinks
Canonicals:Self Referencing Inlinks
Canonicals:Canonicalised Inlinks
Canonicals:Missing Inlinks
Canonicals:Multiple Inlinks
Canonicals:Non-Indexable Canonical Inlinks
AMP:All Inlinks
AMP:Non-200 Response Inlinks
AMP:Missing Non-AMP Return Link Inlinks
AMP:Missing Canonical to Non-AMP Inlinks
AMP:Non-Indexable Canonical Inlinks
AMP:Indexable Inlinks
AMP:Non-Indexable Inlinks
Structured Data:Contains Structured Data
Structured Data:Validation Errors
Structured Data:Validation Warnings
Structured Data:JSON-LD URLs
Structured Data:Microdata URLs
Structured Data:RDFa URLs
Sitemaps:URLs in Sitemap Inlinks
Sitemaps:Orphan URLs Inlinks
Sitemaps:Non-Indexable URLs in Sitemap Inlinks
Sitemaps:URLs in Multiple Sitemaps Inlinks" --save-report "Crawl Overview
Redirects:All Redirects
Redirects:Redirect Chains
Redirects:Redirect & Canonical Chains
Canonicals:Canonical Chains
Canonicals:Non-Indexable Canonicals
Pagination:Non-200 Pagination URLs
Pagination:Unlinked Pagination URLs
Hreflang:All hreflang URLs
Hreflang:Non-200 hreflang URLs
Hreflang:Unlinked hreflang URLs
Hreflang:Missing Return Links
Hreflang:Inconsistent Language & Region Return Links
Hreflang:Non Canonical Return Links
Hreflang:Noindex Return Links
Insecure Content
SERP Summary
Orphan Pages
Structured Data:Validation Errors & Warnings Summary
Structured Data:Validation Errors & Warnings
Structured Data:Google Rich Results Features Summary
Structured Data:Google Rich Results Features
HTTP Headers:HTTP Header Summary
Cookies:Cookie Summary
```

**Summary**:

```
Site Crawled
Date
Time
Total URLs Encountered
Total URLs Crawled
Total Internal blocked by robots.txt
Total External blocked by robots.txt
URLs Displayed
Total Internal URLs
Total External URLs
Total Internal Indexable URLs
Total Internal Non-Indexable URLs
JavaScript:All
JavaScript:Uses Old AJAX Crawling Scheme URLs
JavaScript:Uses Old AJAX Crawling Scheme Meta Fragment Tag
JavaScript:Page Title Only in Rendered HTML
JavaScript:Page Title Updated by JavaScript
JavaScript:H1 Only in Rendered HTML
JavaScript:H1 Updated by JavaScript
JavaScript:Meta Description Only in Rendered HTML
JavaScript:Meta Description Updated by JavaScript
JavaScript:Canonical Only in Rendered HTML
JavaScript:Canonical Mismatch
JavaScript:Noindex Only in Original HTML
JavaScript:Nofollow Only in Original HTML
JavaScript:Contains JavaScript Links
JavaScript:Contains JavaScript Content
JavaScript:Pages with Blocked Resources
H1:All
H1:Missing
H1:Duplicate
H1:Over X Characters
H1:Multiple
H2:All
H2:Missing
H2:Duplicate
H2:Over X Characters
H2:Multiple
Internal:All
Internal:HTML
Internal:JavaScript
Internal:CSS
Internal:Images
Internal:PDF
Internal:Flash
Internal:Other
Internal:Unknown
External:All
External:HTML
External:JavaScript
External:CSS
External:Images
External:PDF
External:Flash
External:Other
External:Unknown
AMP:All
AMP:Non-200 Response
AMP:Missing Non-AMP Return Link
AMP:Missing Canonical to Non-AMP
AMP:Non-Indexable Canonical
AMP:Indexable
AMP:Non-Indexable
AMP:Missing <html amp> Tag
AMP:Missing/Invalid <!doctype html> Tag
AMP:Missing <head> Tag
AMP:Missing <body> Tag
AMP:Missing Canonical
AMP:Missing/Invalid <meta charset> Tag
AMP:Missing/Invalid <meta viewport> Tag
AMP:Missing/Invalid AMP Script
AMP:Missing/Invalid AMP Boilerplate
AMP:Contains Disallowed HTML
AMP:Other Validation Errors
Canonicals:All
Canonicals:Contains Canonical
Canonicals:Self Referencing
Canonicals:Canonicalised
Canonicals:Missing
Canonicals:Multiple
Canonicals:Non-Indexable Canonical
Content:All
Content:Spelling Errors
Content:Grammar Errors
Content:Near Duplicates
Content:Exact Duplicates
Content:Low Content Pages
Custom Extraction:All
Custom Search:All
Directives:All
Directives:Index
Directives:Noindex
Directives:Follow
Directives:Nofollow
Directives:None
Directives:NoArchive
Directives:NoSnippet
Directives:Max-Snippet
Directives:Max-Image-Preview
Directives:Max-Video-Preview
Directives:NoODP
Directives:NoYDIR
Directives:NoImageIndex
Directives:NoTranslate
Directives:Unavailable_After
Directives:Refresh
Analytics:All
Analytics:Sessions Above 0
Analytics:Bounce Rate Above 70%
Analytics:No GA Data
Analytics:Non-Indexable with GA Data
Analytics:Orphan URLs
Search Console:All
Search Console:Clicks Above 0
Search Console:No GSC Data
Search Console:Non-Indexable with GSC Data
Search Console:Orphan URLs
Hreflang:All
Hreflang:Contains hreflang
Hreflang:Non-200 hreflang URLs
Hreflang:Unlinked hreflang URLs
Hreflang:Missing Return Links
Hreflang:Inconsistent Language & Region Return Links
Hreflang:Non-Canonical Return Links
Hreflang:Noindex Return Links
Hreflang:Incorrect Language & Region Codes
Hreflang:Multiple Entries
Hreflang:Missing Self Reference
Hreflang:Not Using Canonical
Hreflang:Missing X-Default
Hreflang:Missing
Images:All
Images:Over X KB
Images:Missing Alt Text
Images:Missing Alt Attribute
Images:Alt Text Over X Characters
Link Metrics:All
Meta Description:All
Meta Description:Missing
Meta Description:Duplicate
Meta Description:Over X Characters
Meta Description:Below X Characters
Meta Description:Over X Pixels
Meta Description:Below X Pixels
Meta Description:Multiple
Meta Keywords:All
Meta Keywords:Missing
Meta Keywords:Duplicate
Meta Keywords:Multiple
PageSpeed:All
PageSpeed:Eliminate Render-Blocking Resources
PageSpeed:Defer Offscreen Images
PageSpeed:Efficiently Encode Images
PageSpeed:Properly Size Images
PageSpeed:Minify CSS
PageSpeed:Minify JavaScript
PageSpeed:Reduce Unused CSS
PageSpeed:Reduce Unused JavaScript
PageSpeed:Serve Images in Next-Gen Formats
PageSpeed:Enable Text Compression
PageSpeed:Preconnect to Required Origins
PageSpeed:Reduce Server Response Times (TTFB)
PageSpeed:Avoid Multiple Page Redirects
PageSpeed:Preload Key Requests
PageSpeed:Use Video Formats for Animated Content
PageSpeed:Avoid Excessive DOM Size
PageSpeed:Reduce JavaScript Execution Time
PageSpeed:Serve Static Assets with an Efficient Cache Policy
PageSpeed:Minimize Main-Thread Work
PageSpeed:Ensure Text Remains Visible During Webfont Load
PageSpeed:Image Elements Do Not Have Explicit Width & Height
PageSpeed:Avoid Large Layout Shifts
PageSpeed:Avoid Serving Legacy JavaScript to Modern Browsers
PageSpeed:Request Errors
Pagination:All
Pagination:Contains Pagination
Pagination:First Page
Pagination:Paginated 2+ Pages
Pagination:Pagination URL Not in Anchor Tag
Pagination:Non-200 Pagination URLs
Pagination:Unlinked Pagination URLs
Pagination:Non-Indexable
Pagination:Multiple Pagination URLs
Pagination:Pagination Loop
Pagination:Sequence Error
Response Codes:All
Response Codes:Blocked by Robots.txt
Response Codes:Blocked Resource
Response Codes:No Response
Response Codes:Success (2xx)
Response Codes:Redirection (3xx)
Response Codes:Redirection (JavaScript)
Response Codes:Redirection (Meta Refresh)
Response Codes:Client Error (4xx)
Response Codes:Server Error (5xx)
Security:All
Security:HTTP URLs
Security:HTTPS URLs
Security:Mixed Content
Security:Form URL Insecure
Security:Form on HTTP URL
Security:Unsafe Cross-Origin Links
Security:Missing HSTS Header
Security:Bad Content Type
Security:Missing X-Content-Type-Options Header
Security:Missing X-Frame-Options Header
Security:Protocol-Relative Resource Links
Security:Missing Content-Security-Policy Header
Security:Missing Secure Referrer-Policy Header
Sitemaps:All
Sitemaps:URLs in Sitemap
Sitemaps:URLs not in Sitemap
Sitemaps:Orphan URLs
Sitemaps:Non-Indexable URLs in Sitemap
Sitemaps:URLs in Multiple Sitemaps
Sitemaps:XML Sitemap with over 50k URLs
Sitemaps:XML Sitemap over 50MB
Structured Data:All
Structured Data:Contains Structured Data
Structured Data:Missing
Structured Data:Validation Errors
Structured Data:Validation Warnings
Structured Data:Parse Errors
Structured Data:Microdata URLs
Structured Data:JSON-LD URLs
Structured Data:RDFa URLs
Page Titles:All
Page Titles:Missing
Page Titles:Duplicate
Page Titles:Over X Characters
Page Titles:Below X Characters
Page Titles:Over X Pixels
Page Titles:Below X Pixels
Page Titles:Same as H1
Page Titles:Multiple
URL:All
URL:Non ASCII Characters
URL:Underscores
URL:Uppercase
URL:Parameters
URL:Over X Characters
URL:Multiple Slashes
URL:Repetitive Path
URL:Contains Space
URL:Broken Bookmark
URL:Internal Search
Depth 1
Depth 2
Depth 3
Depth 4
Depth 5
Depth 6
Depth 7
Depth 8
Depth 9
Depth 10+
Top Inlinks 1 URL
Top Inlinks 1 Number of Inlinks
Top Inlinks 2 URL
Top Inlinks 2 Number of Inlinks
Top Inlinks 3 URL
Top Inlinks 3 Number of Inlinks
Top Inlinks 4 URL
Top Inlinks 4 Number of Inlinks
Top Inlinks 5 URL
Top Inlinks 5 Number of Inlinks
Top Inlinks 6 URL
Top Inlinks 6 Number of Inlinks
Top Inlinks 7 URL
Top Inlinks 7 Number of Inlinks
Top Inlinks 8 URL
Top Inlinks 8 Number of Inlinks
Top Inlinks 9 URL
Top Inlinks 9 Number of Inlinks
Top Inlinks 10 URL
Top Inlinks 10 Number of Inlinks
Top Inlinks 11 URL
Top Inlinks 11 Number of Inlinks
Top Inlinks 12 URL
Top Inlinks 12 Number of Inlinks
Top Inlinks 13 URL
Top Inlinks 13 Number of Inlinks
Top Inlinks 14 URL
Top Inlinks 14 Number of Inlinks
Top Inlinks 15 URL
Top Inlinks 15 Number of Inlinks
Top Inlinks 16 URL
Top Inlinks 16 Number of Inlinks
Top Inlinks 17 URL
Top Inlinks 17 Number of Inlinks
Top Inlinks 18 URL
Top Inlinks 18 Number of Inlinks
Top Inlinks 19 URL
Top Inlinks 19 Number of Inlinks
Top Inlinks 20 URL
Top Inlinks 20 Number of Inlinks
Response Times 0s to 1s
Response Times 1s to 2s
Response Times 2s to 3s
Response Times 3s to 4s
Response Times 4s to 5s
Response Times 5s to 6s
Response Times 6s to 7s
Response Times 7s to 8s
Response Times 8s to 9s
Response Times 10s or more" ```


